## **0. Introduction**

This notebook follows after the "General Linear Model.iypnb" notebook. Aim of this notebook is to conduct the support vector machine. For now, we have 26 z-maps, so one for each contrast, for each session, i.e. 4 * 26 z-maps. We will train the SVM on two sessions and then test the SVM on the remaining two sessions.

## **1. Masking**

First we will need to load the mask. The masks needs to be fetched from the openneuro dataset. This is done with *datalad*.

We cd into the local dataset directory and then use the command:

datalad get sourcedata/sub-01/anat.

The masks are bi-lateral organised, meaning one mask per hemisphere. For starters, the higher visual cortex (HVC) will be the first mask.

In [1]:
import os

os.chdir("/home/jpauli/ds001506/sourcedata/sub-01/anat")
os.listdir()

['sub-01_mask_LH_FFA.nii.gz',
 'sub-01_mask_LH_hV4.nii.gz',
 'sub-01_mask_LH_HVC.nii.gz',
 'sub-01_mask_LH_LOC.nii.gz',
 'sub-01_mask_LH_PPA.nii.gz',
 'sub-01_mask_LH_V1d.nii.gz',
 'sub-01_mask_LH_V1v.nii.gz',
 'sub-01_mask_LH_V2d.nii.gz',
 'sub-01_mask_LH_V2v.nii.gz',
 'sub-01_mask_LH_V3d.nii.gz',
 'sub-01_mask_LH_V3v.nii.gz',
 'sub-01_mask_RH_FFA.nii.gz',
 'sub-01_mask_RH_hV4.nii.gz',
 'sub-01_mask_RH_HVC.nii.gz',
 'sub-01_mask_RH_LOC.nii.gz',
 'sub-01_mask_RH_PPA.nii.gz',
 'sub-01_mask_RH_V1d.nii.gz',
 'sub-01_mask_RH_V1v.nii.gz',
 'sub-01_mask_RH_V2d.nii.gz',
 'sub-01_mask_RH_V2v.nii.gz',
 'sub-01_mask_RH_V3d.nii.gz',
 'sub-01_mask_RH_V3v.nii.gz']

In [2]:
len(os.listdir())

22

We can see that there are 22 masks in total, 11 for each hemisphere. 

In [63]:
mask_img_path = '/home/jpauli/ds001506/sourcedata/sub-01/anat'
mask_img_L = os.path.join(mask_img_path,'sub-01_mask_LH_HVC.nii.gz')
mask_img_R = 'sub-01_mask_RH_HVC.nii.gz'

In [67]:
func_filename_path = '/mnt/c/Users/janos/git/sessions/z_maps_2'
func_filename =  os.path.join(func_filename_path,'02_active -1443537.0_z_map.nii.gz')
os.chdir('/mnt/c/Users/janos/git/sessions/z_maps_2')

In [68]:
func_mean = mean_img(func_filename)

TypeError: 'Nifti1Image' object is not callable

In [65]:
from nilearn.masking import apply_mask
masked_data = apply_mask(func_filename, mask_img_L)

# masked_data shape is (timepoints, voxels). We can plot the first 150
# timepoints from two voxels

# And now plot a few of these
import matplotlib.pyplot as plt
plt.figure(figsize=(7, 5))
plt.plot(masked_data[:150, :2])
plt.xlabel('Time [TRs]', fontsize=16)
plt.ylabel('Intensity', fontsize=16)
plt.xlim(0, 150)
plt.subplots_adjust(bottom=.12, top=.95, right=.95, left=.12)

show()

ValueError: Mask affine: 
[[-1.99986243e+00 -2.03839540e-02 -1.16112223e-02  1.01610313e+02]
 [-1.92472935e-02  1.99166536e+00 -1.81379184e-01 -1.09106407e+02]
 [-1.34115219e-02  1.81254983e-01  1.99172449e+00 -7.27547607e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
 is different from img affine:
[[-2.00000000e+00  0.00000000e+00  6.89032165e-15  9.74524002e+01]
 [ 0.00000000e+00 -1.75182104e+00 -9.64947224e-01  9.18372650e+01]
 [ 0.00000000e+00 -9.64947224e-01  1.75182104e+00  7.57828140e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

In [50]:
os.chdir('/mnt/c/Users/janos/git/sessions/z_maps_2')
a = '02_active -1443537.0_z_map.nii.gz'
#from nilearn.image import mean_img 
#from nilearn.plotting import plot_img
#mean_img = mean_img(a)
#plot_img(mean_img)